In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torchvision
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Q 1-4

# initialize dataset
data = pd.read_csv("C:/Users/haunt/Downloads/diabetes.csv", sep="\t")
sorted_data = data.sort_values(by='Y', ascending=False)
sorted_data['Class'] = pd.qcut(sorted_data['Y'], q=10, labels=False, duplicates='raise') +1 # add Class col


# Split the data into training and test sets
train_data, test_data = train_test_split(sorted_data, test_size=0.2, random_state=42)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y,Class
134,28,1,30.4,85.00,198,115.6,67.0,3.00,4.3438,80,103,4
17,68,2,27.5,111.00,214,147.0,39.0,5.00,4.9416,91,144,6
23,61,2,32.0,103.67,210,85.2,35.0,6.00,6.1070,124,245,9
39,48,2,27.7,73.00,191,119.4,46.0,4.00,4.8520,92,90,3
330,51,2,29.2,107.00,187,139.0,32.0,6.00,4.3820,95,244,9
...,...,...,...,...,...,...,...,...,...,...,...,...
320,42,1,29.2,93.00,249,174.2,45.0,6.00,5.0039,92,122,5
282,68,1,25.9,93.00,253,181.2,53.0,5.00,4.5433,98,230,8
114,55,1,36.6,113.00,199,94.4,43.0,4.63,5.7301,97,258,9
370,54,1,21.0,78.00,188,107.4,70.0,3.00,3.9703,73,63,2


In [19]:
# Q 5
num_of_data_columns = 11
class DiseaseDataset(Dataset):
    
    def __init__(self, data):
        self.df=torch.tensor(data.to_numpy()).float()[:,:num_of_data_columns]
        self.labels=torch.tensor(data['Class'].to_numpy().reshape(-1)).long()

    def __getitem__(self, index):
        return self.df[index],self.labels[index]

    def __len__(self):
        return len(self.df)
    
# Q 6-7
dataset = DiseaseDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size = 10, shuffle = True)

# print one batch
dataiter = iter(train_dataloader)
one_batch = next(dataiter)
features,labels = one_batch
print(features,labels)

# Q 8 - neural network with Y as input
from torch import nn
model = nn.Sequential(
 nn.Linear(11,30),
 nn.Tanh(),
 nn.Linear(30,30),
 nn.Sigmoid(),
 nn.Linear(30,20),
 nn.Sigmoid(),
 nn.Linear(20,10),
 nn.LogSoftmax(dim=1)
 )

# trainning the neural network
CE_loss = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def iterate_batch(features, labels):
    labels = labels -1 
    optimizer.zero_grad()
    y_model = model(features)
    loss = CE_loss(y_model,labels)
    y_model
    loss.backward()
    optimizer.step()
    predicted_labels = y_model.argmax(dim=1)
    acc = (predicted_labels == labels).sum()/len(labels)
    return loss.detach(), acc.detach()

batches = len(train_dataloader)
loss = torch.zeros(batches)
acc = torch.zeros(batches)

epochs = 1001
for i in range(epochs):
    for j, (features, labels) in enumerate(train_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)       
    if i % 100 == 0:
        print(f" epoch = {i}, loss = {loss.mean()}, acc = {acc.mean()}")
        
# Q 10

# testing the net on data test
dataset = DiseaseDataset(test_data)
test_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for j, (features, labels) in enumerate(test_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
print(f"loss = {loss.mean()}, acc = {acc.mean()}")


tensor([[ 28.0000,   2.0000,  24.7000,  97.0000, 175.0000,  99.6000,  32.0000,
           5.0000,   5.3799,  87.0000, 101.0000],
        [ 49.0000,   2.0000,  28.8000,  92.0000, 207.0000, 140.0000,  44.0000,
           5.0000,   4.7449,  92.0000, 196.0000],
        [ 53.0000,   2.0000,  26.6000,  93.0000, 185.0000, 122.4000,  36.0000,
           5.0000,   4.8903,  82.0000, 245.0000],
        [ 44.0000,   1.0000,  25.1000, 133.0000, 182.0000, 113.0000,  55.0000,
           3.0000,   4.2485,  84.0000, 216.0000],
        [ 44.0000,   1.0000,  23.1000,  87.0000, 213.0000, 126.4000,  77.0000,
           3.0000,   3.8712,  72.0000,  52.0000],
        [ 54.0000,   1.0000,  22.6000,  90.0000, 183.0000, 104.2000,  64.0000,
           3.0000,   4.3041,  92.0000,  72.0000],
        [ 34.0000,   1.0000,  31.4000,  87.0000, 149.0000,  93.8000,  46.0000,
           3.0000,   3.8286,  77.0000, 142.0000],
        [ 54.0000,   1.0000,  24.2000,  74.0000, 204.0000, 109.0000,  82.0000,
           2.0000,

In [18]:
# Ajustments for Q 9
num_of_data_columns = 10 # without Y column
class DiseaseDataset(Dataset):
    
    def __init__(self, data):
        self.df=torch.tensor(data.to_numpy()).float()[:,:num_of_data_columns]
        self.labels=torch.tensor(data['Class'].to_numpy().reshape(-1)).long()

    def __getitem__(self, index):
        return self.df[index],self.labels[index]

    def __len__(self):
        return len(self.df)
    
dataset = DiseaseDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size = 10, shuffle = True)


# Q 9 - neural network without Y as input

from torch import nn
model = nn.Sequential(
 nn.Linear(10,30),
 nn.Tanh(),
 nn.Linear(30,30),
 nn.Sigmoid(),
 nn.Linear(30,20),
 nn.Sigmoid(),
 nn.Linear(20,10),
 nn.LogSoftmax(dim=1)
 )

CE_loss = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def iterate_batch(features, labels):
    labels = labels -1
    optimizer.zero_grad()
    y_model = model(features)
    loss = CE_loss(y_model,labels)
    y_model
    loss.backward()
    optimizer.step()
    predicted_labels = y_model.argmax(dim=1)
    acc = (predicted_labels == labels).sum()/len(labels)
    return loss.detach(), acc.detach()


batches = len(train_dataloader)
loss = torch.zeros(batches)
acc = torch.zeros(batches)

epochs = 1001
for i in range(epochs):
    for j, (features, labels) in enumerate(train_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
    if i % 100 == 0:
        print(f" epoch = {i}, loss = {loss.mean()}, acc = {acc.mean()}")
        
# Q 10

dataset = DiseaseDataset(test_data)
test_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for j, (features, labels) in enumerate(test_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
print(f"loss = {loss.mean()}, acc = {acc.mean()}")

 epoch = 0, loss = 2.345939874649048, acc = 0.08611111342906952
 epoch = 100, loss = 2.3008675575256348, acc = 0.11759258806705475
 epoch = 200, loss = 2.3001346588134766, acc = 0.1111111119389534
 epoch = 300, loss = 2.297569751739502, acc = 0.1111111119389534
 epoch = 400, loss = 2.2862982749938965, acc = 0.11388888955116272
 epoch = 500, loss = 2.2291553020477295, acc = 0.15648148953914642
 epoch = 600, loss = 2.2236416339874268, acc = 0.15833333134651184
 epoch = 700, loss = 2.2798190116882324, acc = 0.10925926268100739
 epoch = 800, loss = 2.2910966873168945, acc = 0.12314815074205399
 epoch = 900, loss = 2.291245937347412, acc = 0.12870371341705322
 epoch = 1000, loss = 2.291400909423828, acc = 0.12222222238779068
loss = 2.296696662902832, acc = 0.10864197462797165


In [ ]:
'''
Q 11:

The net that includes Y col has more inputs that helps her to predict correctly the targets.
Also the 'Y' col is the most important input, because the Class col depends mostly on Y col.

'''



In [22]:
# Q 12
sorted_data = data.sort_values(by='Y', ascending=False)
sorted_data['Class'] = pd.qcut(sorted_data['Y'], q=100, labels=False, duplicates='raise') +1 # add Class col


# Split the data into training and test sets
train_data, test_data = train_test_split(sorted_data, test_size=0.2, random_state=42)



num_of_data_columns = 11 # with Y col
class DiseaseDataset(Dataset):
    
    def __init__(self, data):
        self.df=torch.tensor(data.to_numpy()).float()[:,:num_of_data_columns]
        self.labels=torch.tensor(data['Class'].to_numpy().reshape(-1)).long()

    def __getitem__(self, index):
        return self.df[index],self.labels[index]

    def __len__(self):
        return len(self.df)
    


dataset = DiseaseDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size = 10, shuffle = True)

# neural network with Y as input

from torch import nn
model = nn.Sequential(
 nn.Linear(11,30),
 nn.Tanh(),
 nn.Linear(30,30),
 nn.Sigmoid(),
 nn.Linear(30,20),
 nn.Sigmoid(),
 nn.Linear(20,100),
 nn.LogSoftmax(dim=1)
 )

CE_loss = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def iterate_batch(features, labels):
    labels = labels -1 
    optimizer.zero_grad()
    y_model = model(features)
    loss = CE_loss(y_model,labels)
    y_model
    loss.backward()
    optimizer.step()
    predicted_labels = y_model.argmax(dim=1)
    acc = (predicted_labels == labels).sum()/len(labels)
    return loss.detach(), acc.detach()

batches = len(train_dataloader)
loss = torch.zeros(batches)
acc = torch.zeros(batches)

epochs = 1001
for i in range(epochs):
    for j, (features, labels) in enumerate(train_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)       
    if i % 100 == 0:
        print(f" epoch = {i}, loss = {loss.mean()}, acc = {acc.mean()}")
        
# Q 10

dataset = DiseaseDataset(test_data)
test_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for j, (features, labels) in enumerate(test_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
print(f"loss = {loss.mean()}, acc = {acc.mean()}")


 epoch = 0, loss = 4.660356521606445, acc = 0.011111111380159855
 epoch = 100, loss = 4.539266109466553, acc = 0.013888888992369175
 epoch = 200, loss = 4.530793190002441, acc = 0.01944444328546524
 epoch = 300, loss = 4.4937920570373535, acc = 0.03611110895872116
 epoch = 400, loss = 4.332884311676025, acc = 0.03888889029622078
 epoch = 500, loss = 4.119540691375732, acc = 0.0416666716337204
 epoch = 600, loss = 4.014353275299072, acc = 0.0555555559694767
 epoch = 700, loss = 3.9373881816864014, acc = 0.04722222313284874
 epoch = 800, loss = 3.872384786605835, acc = 0.06111111119389534
 epoch = 900, loss = 3.8195486068725586, acc = 0.07037036120891571
 epoch = 1000, loss = 3.7832934856414795, acc = 0.06388887763023376
loss = 3.8555679321289062, acc = 0.0555555559694767


In [24]:
num_of_data_columns = 10
class DiseaseDataset(Dataset):
    
    def __init__(self, data):
        self.df=torch.tensor(data.to_numpy()).float()[:,:num_of_data_columns]
        self.labels=torch.tensor(data['Class'].to_numpy().reshape(-1)).long()

    def __getitem__(self, index):
        return self.df[index],self.labels[index]

    def __len__(self):
        return len(self.df)
    


dataset = DiseaseDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size = 10, shuffle = True)

# neural network without Y as input

from torch import nn
model = nn.Sequential(
 nn.Linear(10,30),
 nn.Tanh(),
 nn.Linear(30,30),
 nn.Sigmoid(),
 nn.Linear(30,20),
 nn.Sigmoid(),
 nn.Linear(20,100),
 nn.LogSoftmax(dim=1)
 )

CE_loss = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def iterate_batch(features, labels):
    labels = labels -1
    optimizer.zero_grad()
    y_model = model(features)
    loss = CE_loss(y_model,labels)
    y_model
    loss.backward()
    optimizer.step()
    predicted_labels = y_model.argmax(dim=1)
    acc = (predicted_labels == labels).sum()/len(labels)
    return loss.detach(), acc.detach()

batches = len(train_dataloader)
loss = torch.zeros(batches)
acc = torch.zeros(batches)

epochs = 1001
for i in range(epochs):
    for j, (features, labels) in enumerate(train_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
    if i % 100 == 0:
        print(f" epoch = {i}, loss = {loss.mean()}, acc = {acc.mean()}")
        
# Q 10

dataset = DiseaseDataset(test_data)
test_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for j, (features, labels) in enumerate(test_dataloader):
        loss[j], acc[j] = iterate_batch(features, labels)
print(f"loss = {loss.mean()}, acc = {acc.mean()}")

 epoch = 0, loss = 4.652468204498291, acc = 0.011111111380159855
 epoch = 100, loss = 4.535065650939941, acc = 0.01666666753590107
 epoch = 200, loss = 4.5302839279174805, acc = 0.01944444514811039
 epoch = 300, loss = 4.532470226287842, acc = 0.01944444328546524
 epoch = 400, loss = 4.533999919891357, acc = 0.01944444514811039
 epoch = 500, loss = 4.530945777893066, acc = 0.01944444514811039
 epoch = 600, loss = 4.528755187988281, acc = 0.01666666753590107
 epoch = 700, loss = 4.525008201599121, acc = 0.01944444514811039
 epoch = 800, loss = 4.522512435913086, acc = 0.01944444514811039
 epoch = 900, loss = 4.482946395874023, acc = 0.02222222276031971
 epoch = 1000, loss = 4.437296390533447, acc = 0.03611111640930176
loss = 4.510091304779053, acc = 0.03333333507180214


In [ ]:
'''
Q 14:

I prefer 10 classes instead of 100 classes. 
One reason is that is much harder to distiguish between 100 classes than 10 classes'
therefore the loss is higher in the 100 classes version.
Second reason is that if I want to adjust my net to 100 outputs, I need much more neurons,
and it will make the time run of my code much longer.

'''